In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Visualize

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import imageio
import os
from tqdm import tqdm

In [ ]:
%cd /content/drive/MyDrive/IDL/kitti/tracking/training/label_02

columns   = 'file frame track_id type truncated occluded alpha bbox_left bbox_top bbox_right bbox_bottom dim_height dim_width dim_length loc_x loc_y loc_z rotation_y'.split()
drop_cols = 'file frame track_id type seq truncated occluded'.split()

ground_truth = np.load('ground_truth.npy')
first_5_frames = np.load('first_5_frames.npy')
predictions = np.load('predicted.npy')

val_frame_data = np.load('val_frame_data_winlen_10.npy', allow_pickle=True)

first_5_frames.shape, ground_truth.shape, predictions.shape, val_frame_data.shape

/content/drive/MyDrive/IDL/kitti/tracking/training/label_02


((675, 5, 12), (675, 5, 12), (675, 5, 12), (675, 10, 7))

In [ ]:
def plot_bbox(img_path, ground_bbox, pred_bbox=None, fname=None):
    img = Image.open(img_path)

    # Create a figure and axes object using matplotlib
    fig, ax = plt.subplots()

    # Display the image on the axes object
    ax.imshow(img)

    # Create a rectangle patch object using the bounding box coordinates
    linewidth = 4
    rect_grnd = patches.Rectangle((ground_bbox[0], ground_bbox[1]), ground_bbox[2]-ground_bbox[0], ground_bbox[3]-ground_bbox[1], linewidth=linewidth, edgecolor='g', facecolor='none')
    if type(pred_bbox) != type(None): rect_pred = patches.Rectangle((pred_bbox[0], pred_bbox[1]), pred_bbox[2]-pred_bbox[0], pred_bbox[3]-pred_bbox[1], linewidth=linewidth, edgecolor='r', facecolor='none')

    # Add the rectangle patch to the axes object
    ax.add_patch(rect_grnd)
    if type(pred_bbox) != type(None): ax.add_patch(rect_pred)

    # # Add labels to the rectangles
    # ax.text(ground_bbox[0], ground_bbox[1]-10, "Ground Truth", fontsize=14, color='g', fontweight='bold')
    # ax.text(pred_bbox[3], pred_bbox[1]-10, "Prediction", fontsize=14, color='r', fontweight='bold')

    # Show the image with the bounding box using matplotlib
    fig.savefig(fname, dpi=2000, bbox_inches='tight')

    plt.close()

In [ ]:
%cd /content/drive/MyDrive/IDL/kitti/tracking/training/
# SAVEDIR = 'visualizations'
SAVEDIR = '/content'
!rm -rf {SAVEDIR}/*.png
!rm -rf {SAVEDIR}/*.gif

for i in tqdm([122]):
    images = []
    # if val_frame_data[i][0][0] != '0019.txt' and val_frame_data[i][0][1]<550: continue
    final_gif_file_name = f'{SAVEDIR}/{val_frame_data[i][0][0].replace(".txt", "")}_{val_frame_data[i][0][1]}_{val_frame_data[i][-1][1]}.gif'
    if os.path.exists(final_gif_file_name): continue
    try: 
        for j in range(5):
            frame = val_frame_data[i][j]
            img_path = f'image_02/{frame[0].replace(".txt","")}/{frame[1]:>06}.png'

            ground_bbox = first_5_frames[i][j][1:5]
            fname   = f'{SAVEDIR}/{frame[0].replace(".txt","")}_{frame[1]:>06}.png'
            plot_bbox(img_path, ground_bbox, None, fname)
            images.append(Image.open(fname))

        for j in range(5, len(val_frame_data[i])):
            frame = val_frame_data[i][j]
            img_path = f'image_02/{frame[0].replace(".txt","")}/{frame[1]:>06}.png'

            ground_bbox = ground_truth[i][j-5][1:5]
            pred_bbox   = predictions [i][j-5][1:5]
            fname   = f'{SAVEDIR}/{frame[0].replace(".txt","")}_{frame[1]:>06}.png'
            plot_bbox(img_path, ground_bbox, pred_bbox, fname)
            images.append(Image.open(fname))

        imageio.mimsave(final_gif_file_name, images, duration=0.2)
    
    except:
        print("Skipping", val_frame_data[i][0][:2])

/content/drive/MyDrive/IDL/kitti/tracking/training


100%|██████████| 1/1 [00:05<00:00,  5.64s/it]


### Misc

In [ ]:
for idx, (frame_data, val_data, ground_data, pred_data) in enumerate(zip(val_frame_data, first_5_frames, ground_truth, predictions)):
    if frame_data[0][0]=='0019.txt' and frame_data[0][1]==673:
        print(frame_data)
        print(val_data[:,5:])
        print(ground_data[:,5:])
        print(pred_data[:,5:])
        print(idx)
        print('-'*20)

[['0019.txt' 673 63 'Car' 464 0 0]
 ['0019.txt' 674 63 'Car' 464 0 0]
 ['0019.txt' 675 63 'Car' 464 0 0]
 ['0019.txt' 676 63 'Car' 464 0 0]
 ['0019.txt' 677 63 'Car' 464 0 0]
 ['0019.txt' 678 63 'Car' 464 0 0]
 ['0019.txt' 679 63 'Car' 464 0 0]
 ['0019.txt' 680 63 'Car' 464 0 0]
 ['0019.txt' 681 63 'Car' 464 0 0]
 ['0019.txt' 682 63 'Car' 464 0 0]]
[[ 1.820924  1.799006  4.401785 -2.836189  1.494133  9.301854  1.607101]
 [ 1.820924  1.799006  4.401785 -2.841016  1.498752  9.095712  1.602117]
 [ 1.820924  1.799006  4.401785 -2.845843  1.503371  8.889571  1.597133]
 [ 1.820924  1.799006  4.401785 -2.85067   1.507989  8.683429  1.592149]
 [ 1.820924  1.799006  4.401785 -2.855496  1.512608  8.477288  1.587165]]
[[ 1.820924  1.799006  4.401785 -2.860323  1.517227  8.271146  1.582182]
 [ 1.820924  1.799006  4.401785 -2.852622  1.514782  8.060742  1.577359]
 [ 1.820924  1.799006  4.401785 -2.844635  1.538819  7.850416  1.572537]
 [ 1.820924  1.799006  4.401785 -2.836792  1.549541  7.640051  1

In [ ]:
aidan_data = np.array([val_data[:,5:],
ground_data[:,5:],
pred_data[:,5:]])
aidan_data.shape
np.save('/content/aidan_data.npy', aidan_data)

first_5_data, ground_data, pred_data = np.load('/content/aidan_data.npy')

In [ ]:
images = []
for frame in range(553, 723):
    images.append(Image.open(f'/content/0019_{frame:>06}.png'))
imageio.mimsave('/content/sample_data/long_merged.gif', images, duration=0.2)